In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

# ==============================================================================
# HÀM ĐỂ TẠO ANIMATION TIME-SERIES CHO MỘT FILE
# ==============================================================================
def animate_timeseries_for_file(mat_file_path, output_gif_path,
                                sample_length=4096, num_samples_for_animation=10, fps=4):
    """
    Tải dữ liệu từ file .mat, và tạo animation time-series (GIF) cho nhiều sample.

    Args:
        mat_file_path (str): Đường dẫn đến file .mat.
        output_gif_path (str): Đường dẫn để lưu file GIF output.
        sample_length (int): Số điểm dữ liệu trong một sample (frame của animation).
        num_samples_for_animation (int): Số lượng sample (frame) sẽ được dùng để tạo animation.
        fps (int): Số frame trên giây cho GIF.
    """
    try:
        mat_data = scipy.io.loadmat(mat_file_path)
        file_basename = os.path.basename(mat_file_path)
        print(f"\n--- Tạo animation time-series cho file: {file_basename} ---")

        # --- Logic tìm data_key (tương tự các hàm đã có) ---
        data_key = None
        keys_in_file = list(mat_data.keys())
        data_keys_info = {}
        for key in keys_in_file:
            if not key.startswith('__'): # Bỏ qua các key metadata của scipy
                try:
                    data_array = mat_data[key]
                    if data_array.size > 1: # Chỉ quan tâm đến các array có dữ liệu
                        data_keys_info[key] = {'shape': data_array.shape, 'size': data_array.size}
                except Exception:
                    pass # Bỏ qua nếu không truy cập được

        # Ưu tiên các key chứa DE, FE, BA (Drive End, Fan End, Base Accelerometer)
        priority_suffixes = ["_DE_time", "_FE_time", "_BA_time"]
        # Lấy phần tên file không có đuôi, ví dụ "118_0" từ "118_0.mat"
        actual_file_id_part = os.path.splitext(os.path.basename(mat_file_path))[0]

        try:
            # Cố gắng trích xuất phần số từ ID file, ví dụ '118' từ '118_0'
            numeric_part_of_id = ''.join(filter(str.isdigit, actual_file_id_part.split('_')[0]))
            if numeric_part_of_id:
                 # Tạo các key tiềm năng dựa trên ID, ví dụ "X118" hoặc "X118_0"
                 potential_file_based_keys = [f"X{int(numeric_part_of_id):03d}", f"X{actual_file_id_part}"]
            else: # Nếu không có phần số (ví dụ: "normal")
                 potential_file_based_keys = [f"X{actual_file_id_part}"]
        except ValueError: # Xử lý trường hợp tên file không theo chuẩn số
            potential_file_based_keys = [f"X{actual_file_id_part}"]

        # Tìm key dựa trên sự kết hợp của ID file và suffix ưu tiên
        for suffix in priority_suffixes:
            for k_check in data_keys_info.keys():
                for pk_base in potential_file_based_keys:
                    if pk_base in k_check and suffix in k_check:
                        data_key = k_check
                        break
                if data_key: break
            if data_key: break
        
        # Nếu không tìm thấy với suffix, thử tìm chỉ với ID file
        if data_key is None:
            for pk in potential_file_based_keys:
                if pk in data_keys_info.keys():
                    data_key = pk
                    break
        
        # Nếu vẫn không tìm thấy, dùng key có nhiều dữ liệu nhất (trừ 'RPM')
        if data_key is None:
            largest_key, max_size = None, 0
            for k, info in data_keys_info.items():
                if k.upper() != 'RPM' and info['size'] > max_size:
                    max_size, largest_key = info['size'], k
            if largest_key:
                data_key = largest_key
                print(f"Cảnh báo: Không tìm thấy key DE/FE/BA hoặc key dựa trên tên file. Sử dụng key có nhiều dữ liệu nhất: '{data_key}'")
            else:
                print(f"Lỗi: Không tìm thấy key dữ liệu phù hợp trong {file_basename}.")
                return
        # --- Kết thúc logic tìm data_key ---

        print(f"Đã chọn key dữ liệu: '{data_key}'")
        time_series_data = mat_data[data_key].flatten() # Lấy dữ liệu và làm phẳng thành mảng 1D
        total_data_points = len(time_series_data)

        if total_data_points == 0 or sample_length <= 0 or num_samples_for_animation <= 0:
            print("Dữ liệu không hợp lệ hoặc không có sample để tạo animation.")
            return

        all_time_samples_for_anim = [] # Danh sách chứa các sample (frame)
        num_possible_full_samples = total_data_points // sample_length
        actual_num_frames = 0 # Số frame thực tế sẽ tạo

        if num_possible_full_samples == 0: # Nếu tổng dữ liệu còn ít hơn 1 sample_length
            if total_data_points > 0 :
                all_time_samples_for_anim.append(time_series_data[:]) # Dùng toàn bộ tín hiệu làm 1 frame
                actual_num_frames = 1
                print(f"Cảnh báo: Tín hiệu quá ngắn ({total_data_points} điểm), tạo animation với 1 frame (toàn bộ tín hiệu).")
        else:
            actual_num_frames = min(num_samples_for_animation, num_possible_full_samples)
            if num_samples_for_animation > num_possible_full_samples:
                print(f"Cảnh báo: Yêu cầu {num_samples_for_animation} frame, nhưng chỉ có {num_possible_full_samples} sample đầy đủ.")
            print(f"Sẽ tạo animation time-series với {actual_num_frames} frame(s).")

            for i in range(actual_num_frames):
                start_idx = i * sample_length
                end_idx = start_idx + sample_length
                all_time_samples_for_anim.append(time_series_data[start_idx:end_idx])

        if not all_time_samples_for_anim: # Không có frame nào được chuẩn bị
            print(f"Không có đủ dữ liệu trong {file_basename} để tạo animation time-series.")
            return

        # Xác định min/max biên độ tổng thể cho tất cả các frame để trục Y cố định
        min_overall_amplitude = float('inf')
        max_overall_amplitude = float('-inf')
        # Độ dài sample cho trục X (thường là sample_length, trừ khi frame duy nhất ngắn hơn)
        current_plot_sample_length = sample_length 

        for i, time_sample_data in enumerate(all_time_samples_for_anim):
            if len(time_sample_data) > 0:
                min_overall_amplitude = min(min_overall_amplitude, np.min(time_sample_data))
                max_overall_amplitude = max(max_overall_amplitude, np.max(time_sample_data))
            if i == 0: # Lấy độ dài của sample đầu tiên để thiết lập xlim ban đầu
                current_plot_sample_length = len(time_sample_data)


        if min_overall_amplitude == float('inf'): min_overall_amplitude = -0.1 # Giá trị mặc định nếu không có dữ liệu
        if max_overall_amplitude == float('-inf'): max_overall_amplitude = 0.1  # Giá trị mặc định
        if min_overall_amplitude == max_overall_amplitude: # Tránh trục y bị thu gọn nếu tín hiệu phẳng
            min_overall_amplitude -= 0.1
            max_overall_amplitude += 0.1

        # Thiết lập figure và axes cho animation
        fig, ax = plt.subplots(figsize=(10, 5)) 
        line, = ax.plot([], [], lw=1.5) # Đường line sẽ được cập nhật trong mỗi frame

        # Hàm khởi tạo cho animation (vẽ frame đầu tiên và thiết lập đồ thị)
        def init_animation_td():
            # Thiết lập giới hạn trục X dựa trên độ dài sample
            # (current_plot_sample_length là độ dài của frame đầu tiên, hoặc frame duy nhất nếu tín hiệu ngắn)
            ax.set_xlim([0, current_plot_sample_length -1 if current_plot_sample_length > 0 else sample_length -1])
            
            # Thêm một chút padding cho trục Y
            y_padding_min = 0.1 * abs(min_overall_amplitude) if min_overall_amplitude != 0 else 0.1
            y_padding_max = 0.1 * abs(max_overall_amplitude) if max_overall_amplitude != 0 else 0.1
            ax.set_ylim([min_overall_amplitude - y_padding_min,
                         max_overall_amplitude + y_padding_max])
            
            ax.set_xlabel(f"Chỉ số điểm dữ liệu (trong sample, tối đa {sample_length-1} nếu sample đủ dài)")
            ax.set_ylabel("Biên độ gia tốc")
            ax.grid(True)
            line.set_data([], []) # Xóa dữ liệu cũ của line
            return line,

        # Hàm cập nhật cho mỗi frame của animation
        def update_animation_td(frame_index):
            time_sample_data = all_time_samples_for_anim[frame_index] # Lấy dữ liệu cho frame hiện tại
            current_N_points = len(time_sample_data) # Số điểm trong sample hiện tại
            sample_indices = np.arange(current_N_points) # Tạo trục X cho sample
            
            line.set_data(sample_indices, time_sample_data) # Cập nhật dữ liệu cho line
            
            # Điều chỉnh xlim nếu frame hiện tại có độ dài khác (ví dụ: frame duy nhất ngắn)
            if current_N_points > 0:
                 ax.set_xlim([0, current_N_points - 1])
            else: 
                 ax.set_xlim([0, sample_length -1]) # Mặc định nếu frame rỗng

            start_point_of_sample = frame_index * sample_length # Điểm bắt đầu của sample trong tín hiệu gốc
            ax.set_title(f"Time Domain - File: {file_basename} - Sample {frame_index}\n"
                         f"(Key: {data_key}, Điểm bắt đầu: {start_point_of_sample}, {current_N_points} điểm)")
            return line,

        if actual_num_frames > 0:
            # Tạo đối tượng animation
            anim = FuncAnimation(fig, update_animation_td, frames=actual_num_frames,
                                 init_func=init_animation_td, blit=True, interval=1000/fps)
            try:
                # Đảm bảo thư mục đích của file GIF tồn tại
                output_dir_for_gif = os.path.dirname(output_gif_path)
                if not os.path.exists(output_dir_for_gif) and output_dir_for_gif: # Kiểm tra output_dir_for_gif không rỗng
                    os.makedirs(output_dir_for_gif, exist_ok=True)
                    print(f"Đã tạo thư mục: {output_dir_for_gif}")

                anim.save(output_gif_path, writer='pillow', fps=fps) # Lưu animation
                print(f"Đã lưu animation time-series vào: {output_gif_path}")
            except Exception as e_save:
                print(f"Lỗi khi lưu animation time-series: {e_save}.")
                print("Hãy đảm bảo Pillow (`pip install Pillow`) đã được cài đặt và đường dẫn hợp lệ.")
        else:
            print("Không có frame nào được tạo cho animation time-series.")
        
        plt.close(fig) # Đóng figure sau khi lưu hoặc nếu không có frame để tránh hiển thị thừa

    except FileNotFoundError:
        print(f"Lỗi: File không tìm thấy tại '{mat_file_path}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý file {os.path.basename(mat_file_path)} cho animation time-series: {e}")
        import traceback
        traceback.print_exc()

# ==============================================================================
# PHẦN MAIN ĐỂ XỬ LÝ NHIỀU FILE (TẠO ANIMATION TIME-SERIES)
# ==============================================================================
if __name__ == "__main__":
    # --- Cấu hình ---
    # THAY ĐỔI ĐƯỜNG DẪN NÀY tới thư mục gốc chứa dữ liệu CWRU của bạn
    # Ví dụ: base_data_directory = "D:/CWRU_Dataset/12k_Drive_End_Bearing_Fault_Data/"
    base_data_directory = "CWRU-dataset-main/12k_Drive_End_Bearing_Fault_Data/" # Giả sử thư mục này cùng cấp với script hoặc bạn cung cấp đường dẫn tuyệt đối

    # Thư mục lưu GIF time-series, sẽ được tạo trong thư mục chạy script nếu là đường dẫn tương đối
    # Ví dụ: "./td_animations" sẽ tạo thư mục "td_animations" tại nơi bạn chạy script
    output_td_animation_directory = "./td_animations_output1" 
    os.makedirs(output_td_animation_directory, exist_ok=True) # Tạo thư mục nếu chưa có

    # Danh sách các file bạn muốn xử lý (đường dẫn tương đối từ base_data_directory)
    # Cập nhật danh sách này với các file bạn muốn tạo animation
    files_to_process_td = [
        "B/007/118_0.mat",   # Normal Baseline
        "B/014/185_0.mat",  # Inner Race Fault, 0.007", 0HP
        "B/021/222_0.mat",  # Ball Fault, 0.007", 0HP
        "IR/007/105_0.mat", # Inner Race Fault, 0.007", 0HP
        "IR/014/169_0.mat", # Inner Race Fault, 0.007", 1HP
        "IR/021/209_0.mat", # Ball Fault, 0.007", 1HP
        "OR/007/@6/130@6_0.mat", # Outer Race Fault (@6:00), 0.007", 0HP 
        "OR/014/197@6_0.mat", # Outer Race Fault (@6:00), 0.007", 2HP
        "OR/021/@6/234_0.mat", # Outer Race Fault (@6:00), 0.007", 2HP
        "Normal/97_Normal_0.mat",     # Normal (thường nằm trong thư mục con "Normal" hoặc trực tiếp)
        # Thêm các file khác vào đây nếu muốn
        # "OR/007/@3/144_0.mat", # Ví dụ file khác
    ]

    num_samples_for_td_anim_per_file = 15 # Số frame (sample) cho mỗi animation GIF
    sample_len_td = 2048                  # Độ dài của mỗi sample (frame)
    animation_td_fps = 3                  # Số frame trên giây cho GIF

    print(f"Bắt đầu xử lý các file từ thư mục: '{os.path.abspath(base_data_directory)}'")
    print(f"Sẽ tạo animation time-series với {num_samples_for_td_anim_per_file} frame(s) cho mỗi file (nếu có đủ dữ liệu).\n")

    processed_td_files_count = 0
    failed_td_files_count = 0

    for file_name_relative in files_to_process_td:
        # Đường dẫn đầy đủ đến file .mat
        full_file_path = os.path.join(base_data_directory, file_name_relative)

        # Tạo tên file GIF output, giữ nguyên cấu trúc thư mục con trong thư mục output
        # Ví dụ: file_name_relative = "B/007/118_0.mat"
        # file_name_relative_no_ext = "B/007/118_0"
        file_name_relative_no_ext = os.path.splitext(file_name_relative)[0]
        
        # Tên file GIF sẽ là: "B/007/118_0_td_anim_15frames.gif" (nằm trong output_td_animation_directory)
        gif_sub_path_and_name = f"{file_name_relative_no_ext}_td_anim_{num_samples_for_td_anim_per_file}frames.gif"
        full_gif_output_path = os.path.join(output_td_animation_directory, gif_sub_path_and_name)
        
        # Đảm bảo thư mục con cho file GIF hiện tại tồn tại
        # Ví dụ: nếu full_gif_output_path là "td_animations_output/B/007/118_0_...gif",
        # thì thư mục "td_animations_output/B/007/" cần được tạo.
        gif_output_dir_for_current_file = os.path.dirname(full_gif_output_path)
        if not os.path.exists(gif_output_dir_for_current_file) and gif_output_dir_for_current_file:
             os.makedirs(gif_output_dir_for_current_file, exist_ok=True)


        print(f"=============================================================")
        print(f"ĐANG XỬ LÝ FILE (Time-Series Anim): {file_name_relative}")
        print(f"Đường dẫn đầy đủ: {full_file_path}")
        print(f"GIF output dự kiến: {full_gif_output_path}")
        print(f"=============================================================")

        if os.path.exists(full_file_path):
            try:
                animate_timeseries_for_file(
                    mat_file_path=full_file_path,
                    output_gif_path=full_gif_output_path,
                    sample_length=sample_len_td,
                    num_samples_for_animation=num_samples_for_td_anim_per_file,
                    fps=animation_td_fps
                )
                print(f"Hoàn tất animation time-series cho: {file_name_relative}")
                processed_td_files_count += 1
            except Exception as e_main:
                print(f"Lỗi không mong muốn khi gọi animate_timeseries_for_file cho {file_name_relative}: {e_main}")
                failed_td_files_count += 1
        else:
            print(f"LỖI: File không tồn tại: {full_file_path}. Bỏ qua.")
            failed_td_files_count += 1
        print("-------------------------------------------------------------\n")

    print("=================== TỔNG KẾT (Time-Series Animation) ===================")
    print(f"Đã hoàn tất quá trình xử lý {len(files_to_process_td)} file được chỉ định.")
    print(f"Số file được xử lý thành công (đã gọi hàm tạo animation): {processed_td_files_count}")
    print(f"Số file không tìm thấy hoặc gặp lỗi: {failed_td_files_count}")
    print(f"Các file GIF time-series (nếu có) được lưu trong thư mục: '{os.path.abspath(output_td_animation_directory)}'")
    print("========================================================================")

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import os # Để làm việc với đường dẫn file

# ==============================================================================
# ĐỊNH NGHĨA HÀM plot_cwru_sample (lấy từ câu trả lời trước)
# ==============================================================================
def plot_cwru_sample(mat_file_path, sample_length=4096, num_samples_to_plot=1):
    """
    Tải dữ liệu từ một file .mat của CWRU, in thông tin chung và
    vẽ một số lượng sample cụ thể trong miền thời gian, bắt đầu từ sample 0.

    Args:
        mat_file_path (str): Đường dẫn đến file .mat.
        sample_length (int): Số điểm dữ liệu trong một sample.
        num_samples_to_plot (int): Số lượng sample muốn vẽ, bắt đầu từ sample 0.
                                     Mặc định là 1 (vẽ sample đầu tiên).
    """
    try:
        # Tải file .mat
        mat_data = scipy.io.loadmat(mat_file_path)
        print(f"--- Thông tin chung của file: {os.path.basename(mat_file_path)} ---")

        # 1. In tất cả các key có trong file
        print("\n1. Các key (biến) có trong file .mat:")
        keys_in_file = list(mat_data.keys())
        for key in keys_in_file:
            print(f"   - Key: '{key}'")

        # 2. In kích thước và số lượng phần tử cho mỗi key (trừ các key metadata)
        print("\n2. Kích thước và số lượng phần tử của từng biến dữ liệu:")
        data_keys_info = {}
        for key in keys_in_file:
            if not key.startswith('__'):
                try:
                    data_array = mat_data[key]
                    shape = data_array.shape
                    size = data_array.size
                    dtype = data_array.dtype
                    print(f"   - Key: '{key}', Kiểu dữ liệu: {dtype}, Kích thước (Shape): {shape}, Tổng số phần tử: {size}")
                    if size > 1:
                        data_keys_info[key] = {'shape': shape, 'size': size}
                except Exception as e:
                    print(f"   - Key: '{key}', Không thể truy cập thông tin chi tiết: {e}")

        data_key = None
        priority_suffixes = ["_FE_time", "_DE_time", "_BA_time"]
        basename_no_ext = os.path.splitext(os.path.basename(mat_file_path))[0]
        try:
            potential_file_based_keys = [f"X{int(basename_no_ext):03d}", f"X{basename_no_ext}"]
        except ValueError: # Nếu tên file không phải là số (ví dụ "normal.mat")
            potential_file_based_keys = [f"X{basename_no_ext}"]


        for suffix in priority_suffixes:
            for k in data_keys_info.keys():
                if suffix in k:
                    data_key = k
                    break
            if data_key:
                break
        if data_key is None:
            for pk in potential_file_based_keys:
                if pk in data_keys_info.keys():
                    data_key = pk
                    break
        if data_key is None:
            largest_key = None
            max_size = 0
            for k, info in data_keys_info.items():
                if k.upper() != 'RPM' and info['size'] > max_size:
                    max_size = info['size']
                    largest_key = k
            if largest_key:
                data_key = largest_key
                print(f"Không tìm thấy key DE/FE/BA hoặc key dựa trên tên file. Sử dụng key có nhiều dữ liệu nhất: '{data_key}'")
            else:
                print("Lỗi: Không tìm thấy key dữ liệu phù hợp trong file .mat.")
                return

        print(f"\nĐã chọn key dữ liệu để xử lý: '{data_key}'")
        time_series_data = mat_data[data_key].flatten()
        total_data_points = len(time_series_data)
        print(f"   - Tổng số điểm dữ liệu trong key '{data_key}': {total_data_points}")

        if total_data_points == 0:
            print("Lỗi: Không có điểm dữ liệu nào trong key đã chọn.")
            return
        if sample_length <= 0:
            print("Lỗi: sample_length phải là số dương.")
            return
        if num_samples_to_plot <= 0:
            print("Lỗi: Số lượng sample cần vẽ (num_samples_to_plot) phải là một số dương.")
            return

        num_possible_full_samples = total_data_points // sample_length
        remaining_points = total_data_points % sample_length
        print(f"\n3. Thông tin về sample (với sample_length = {sample_length}):")
        print(f"   - Số lượng sample đầy đủ có thể tạo ra: {num_possible_full_samples}")
        if remaining_points > 0:
            print(f"   - Số điểm dữ liệu còn lại (không đủ cho một sample đầy đủ): {remaining_points}")

        print("--- Kết thúc thông tin chung ---\n")

        actual_num_to_plot = 0
        plot_entire_signal_as_one_sample = False

        if num_possible_full_samples == 0:
            if total_data_points > 0 :
                print(f"Cảnh báo: Độ dài tín hiệu ({total_data_points}) nhỏ hơn sample_length ({sample_length}).")
                print(f"Sẽ vẽ toàn bộ {total_data_points} điểm dữ liệu có sẵn như một sample duy nhất (nếu num_samples_to_plot > 0).")
                actual_num_to_plot = 1
                plot_entire_signal_as_one_sample = True
            else:
                print("Không có dữ liệu để vẽ.")
                return
        else:
            if num_samples_to_plot > num_possible_full_samples:
                print(f"Cảnh báo: Yêu cầu vẽ {num_samples_to_plot} sample, nhưng chỉ có {num_possible_full_samples} sample đầy đủ.")
                print(f"Sẽ vẽ {num_possible_full_samples} sample có sẵn, bắt đầu từ sample 0.")
                actual_num_to_plot = num_possible_full_samples
            else:
                actual_num_to_plot = num_samples_to_plot
                print(f"Sẽ vẽ {actual_num_to_plot} sample, bắt đầu từ sample 0.")

        if actual_num_to_plot == 0:
            print("Không có sample nào để vẽ dựa trên các điều kiện đã cho.")
            return

        for i in range(actual_num_to_plot):
            current_sample_index_to_display = i
            single_sample_data = []
            actual_start_index = 0
            current_sample_actual_length = 0

            if plot_entire_signal_as_one_sample:
                single_sample_data = time_series_data[:]
                actual_start_index = 0
                current_sample_actual_length = len(single_sample_data)
                current_sample_index_to_display = 0
                print(f"\n--- Đang vẽ Sample duy nhất (toàn bộ tín hiệu ngắn) ---")
                print(f"Trích xuất toàn bộ {current_sample_actual_length} điểm dữ liệu.")
            else:
                actual_start_index = current_sample_index_to_display * sample_length
                end_index = actual_start_index + sample_length
                end_index = min(end_index, total_data_points)
                single_sample_data = time_series_data[actual_start_index:end_index]
                current_sample_actual_length = len(single_sample_data)

                if current_sample_actual_length == 0:
                    print(f"Không có dữ liệu cho sample {current_sample_index_to_display} tại vị trí {actual_start_index} (bỏ qua).")
                    continue
                print(f"\n--- Đang vẽ Sample {current_sample_index_to_display} ---")
                print(f"Trích xuất điểm từ {actual_start_index} đến {actual_start_index + current_sample_actual_length - 1}.")

            sample_indices = np.arange(current_sample_actual_length)
            xlabel_text = f"Chỉ số điểm dữ liệu (trong sample, bắt đầu từ điểm {actual_start_index} của tín hiệu gốc)"

            plt.figure(figsize=(7.5, 3.5))
            plt.plot(sample_indices, single_sample_data)
            title_str = (f"Time Domain - Sample {current_sample_index_to_display} từ File: {os.path.basename(mat_file_path)}\n"
                         f"(Key: {data_key}, {current_sample_actual_length} điểm)")
            plt.title(title_str)
            plt.xlabel(xlabel_text)
            plt.ylabel("Biên độ gia tốc")
            plt.grid(True)
            plt.tight_layout()
            plt.show() # Hiển thị đồ thị cho mỗi sample

    except FileNotFoundError:
        print(f"Lỗi: File không tìm thấy tại '{mat_file_path}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi không mong muốn khi xử lý file {os.path.basename(mat_file_path)}: {e}")
        import traceback
        traceback.print_exc()

# ==============================================================================
# PHẦN MAIN ĐỂ XỬ LÝ NHIỀU FILE
# ==============================================================================
if __name__ == "__main__":
    # --- Cấu hình ---
    # THAY ĐỔI ĐƯỜNG DẪN NÀY tới thư mục chứa các file .mat của bạn
    base_data_directory = "CWRU-dataset-main/12k_Drive_End_Bearing_Fault_Data/"  # Giả sử các file .mat nằm cùng thư mục với script
    # Ví dụ: base_data_directory = "/mnt/data/CWRU_Dataset/12k_Drive_End_Bearing_Fault_Data/"

    # Danh sách TÊN các file bạn muốn xử lý (không bao gồm đường dẫn thư mục)
    # Hãy cập nhật danh sách này với 9 (hoặc nhiều hơn) file bạn muốn
    file_names_to_process = [
        "B/007/118_0.mat",   # Normal Baseline
        "B/014/185_0.mat",  # Inner Race Fault, 0.007", 0HP
        "B/021/222_0.mat",  # Ball Fault, 0.007", 0HP
        "IR/007/105_0.mat",  # Outer Race Fault (@6:00), 0.007", 0HP
        "IR/014/169_0.mat",  # Inner Race Fault, 0.007", 1HP
        "IR/021/209_0.mat",  # Ball Fault, 0.007", 1HP
        "OR/007/@3/144_0.mat",  # Outer Race Fault (@6:00), 0.007", 1HP
        "OR/014/197@6_0.mat",  # Inner Race Fault, 0.007", 2HP
        "OR/021/@3/246_0.mat",  # Ball Fault, 0.007", 2HP
        "97_Normal_0.mat",  # Outer Race Fault (@6:00), 0.007", 2HP
        # "98.mat", # Normal
        # "106.mat", # IR007 at 1HP (khác 169)
        # Thêm các file khác vào đây
    ]

    # Số lượng sample đầu tiên bạn muốn vẽ cho mỗi file
    num_samples_to_draw_per_file = 5
    # Độ dài của mỗi sample
    sample_len = 4096

    # --- Xử lý các file ---
    print(f"Bắt đầu xử lý các file từ thư mục (hoặc thư mục hiện tại): '{os.path.abspath(base_data_directory)}'")
    print(f"Sẽ cố gắng vẽ {num_samples_to_draw_per_file} sample(s) đầu tiên cho mỗi file (nếu có đủ dữ liệu).\n")

    processed_files_count = 0
    failed_files_count = 0

    for file_name in file_names_to_process:
        full_file_path = os.path.join(base_data_directory, file_name)

        print(f"=============================================================")
        print(f"ĐANG XỬ LÝ FILE: {file_name} (Đường dẫn: {full_file_path})")
        print(f"=============================================================")

        if os.path.exists(full_file_path):
            try:
                plot_cwru_sample(
                    mat_file_path=full_file_path,
                    sample_length=sample_len,
                    num_samples_to_plot=num_samples_to_draw_per_file
                )
                print(f"\nHoàn tất xử lý và vẽ đồ thị cho file: {file_name}")
                processed_files_count += 1
            except Exception as e:
                print(f"Lỗi không mong muốn xảy ra khi gọi plot_cwru_sample cho {file_name}: {e}")
                failed_files_count +=1
        else:
            print(f"LỖI: File không tồn tại tại đường dẫn: {full_file_path}")
            print(f"Bỏ qua file: {file_name}\n")
            failed_files_count += 1

        print("-------------------------------------------------------------\n")

    print("=================== TỔNG KẾT ===================")
    print(f"Đã hoàn tất quá trình xử lý {len(file_names_to_process)} file được chỉ định.")
    print(f"Số file được xử lý thành công (đã gọi hàm vẽ): {processed_files_count}")
    print(f"Số file không tìm thấy hoặc gặp lỗi trước khi vẽ: {failed_files_count}")
    print("================================================")

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import os

# ==============================================================================
# HÀM ĐỂ VẼ CHỈ PHỔ TẦN SỐ (FFT)
# ==============================================================================
def plot_cwru_fft_only(mat_file_path, sampling_rate, sample_length=4096, num_samples_to_plot=1):
    """
    Tải dữ liệu từ một file .mat của CWRU và vẽ chỉ phổ tần số (FFT)
    cho một số lượng sample cụ thể, bắt đầu từ sample 0.

    Args:
        mat_file_path (str): Đường dẫn đến file .mat.
        sampling_rate (float): Tần số lấy mẫu của dữ liệu (Hz). Ví dụ: 12000 hoặc 48000.
        sample_length (int): Số điểm dữ liệu trong một sample.
        num_samples_to_plot (int): Số lượng sample muốn vẽ, bắt đầu từ sample 0.
    """
    try:
        mat_data = scipy.io.loadmat(mat_file_path)
        print(f"\n--- Xử lý file: {os.path.basename(mat_file_path)} (Fs = {sampling_rate} Hz) ---")

        # Tìm key dữ liệu (logic tương tự các hàm trước)
        data_key = None
        keys_in_file = list(mat_data.keys())
        data_keys_info = {}
        for key in keys_in_file:
            if not key.startswith('__'):
                try:
                    data_array = mat_data[key]
                    if data_array.size > 1:
                         data_keys_info[key] = {'shape': data_array.shape, 'size': data_array.size}
                except Exception:
                    pass

        priority_suffixes = ["_FE_time", "_DE_time", "_BA_time"]
        basename_no_ext = os.path.splitext(os.path.basename(mat_file_path))[0]
        try:
            potential_file_based_keys = [f"X{int(basename_no_ext):03d}", f"X{basename_no_ext}"]
        except ValueError:
            potential_file_based_keys = [f"X{basename_no_ext}"]

        for suffix in priority_suffixes:
            for k in data_keys_info.keys():
                if suffix in k:
                    data_key = k
                    break
            if data_key: break
        if data_key is None:
            for pk in potential_file_based_keys:
                if pk in data_keys_info.keys():
                    data_key = pk
                    break
        if data_key is None:
            largest_key = None
            max_size = 0
            for k, info in data_keys_info.items():
                if k.upper() != 'RPM' and info['size'] > max_size:
                    max_size = info['size']
                    largest_key = k
            if largest_key: data_key = largest_key
            else:
                print(f"Lỗi: Không tìm thấy key dữ liệu phù hợp trong file {os.path.basename(mat_file_path)}.")
                return

        print(f"Đã chọn key dữ liệu: '{data_key}'")
        time_series_data = mat_data[data_key].flatten()
        total_data_points = len(time_series_data)
        print(f"Tổng số điểm dữ liệu trong key '{data_key}': {total_data_points}")

        if total_data_points == 0 or sample_length <= 0 or num_samples_to_plot <= 0:
            print("Dữ liệu không hợp lệ hoặc không có sample để vẽ.")
            return

        num_possible_full_samples = total_data_points // sample_length
        actual_num_to_plot = 0
        plot_entire_signal_as_one_sample = False

        if num_possible_full_samples == 0:
            if total_data_points > 0:
                print(f"Cảnh báo: Độ dài tín hiệu ({total_data_points}) < sample_length ({sample_length}).")
                print(f"Sẽ vẽ FFT cho toàn bộ tín hiệu như 1 sample duy nhất.")
                actual_num_to_plot = 1
                plot_entire_signal_as_one_sample = True
            else: return
        else:
            actual_num_to_plot = min(num_samples_to_plot, num_possible_full_samples)
            if num_samples_to_plot > num_possible_full_samples:
                print(f"Cảnh báo: Yêu cầu vẽ FFT cho {num_samples_to_plot} sample, nhưng chỉ có {num_possible_full_samples} sample đầy đủ.")
            print(f"Sẽ vẽ FFT cho {actual_num_to_plot} sample(s), bắt đầu từ sample 0.")

        if actual_num_to_plot == 0:
            print("Không có sample nào để vẽ FFT.")
            return

        for i in range(actual_num_to_plot):
            current_sample_index_to_display = i
            single_time_sample = []
            actual_start_index = 0
            current_sample_actual_length = 0

            if plot_entire_signal_as_one_sample:
                single_time_sample = time_series_data[:]
                actual_start_index = 0
                current_sample_actual_length = len(single_time_sample)
                current_sample_index_to_display = 0
                print(f"\n--- Đang xử lý FFT cho Sample duy nhất (toàn bộ tín hiệu) ---")
            else:
                actual_start_index = current_sample_index_to_display * sample_length
                end_index = actual_start_index + sample_length
                single_time_sample = time_series_data[actual_start_index:end_index]
                current_sample_actual_length = len(single_time_sample)
                print(f"\n--- Đang xử lý FFT cho Sample {current_sample_index_to_display} (Điểm {actual_start_index} - {end_index-1}) ---")

            if current_sample_actual_length == 0:
                continue

            # --- Tính toán FFT ---
            N = current_sample_actual_length
            window = np.hanning(N) # Cửa sổ Hanning
            y_windowed = single_time_sample * window

            yf = np.fft.fft(y_windowed)
            xf = np.fft.fftfreq(N, d=1/sampling_rate)

            yf_magnitude = np.abs(yf[:N//2]) * (2.0/N)
            if N > 0: # Tránh lỗi chia cho 0 nếu N = 0 (mặc dù đã kiểm tra current_sample_actual_length)
                 yf_magnitude[0] = np.abs(yf[0]) / N if N > 0 else 0
            xf_positive = xf[:N//2]

            # --- Vẽ đồ thị (CHỈ Frequency Domain) ---
            plt.figure(figsize=(12, 6)) # Kích thước cho một đồ thị đơn

            plt.plot(xf_positive, yf_magnitude)
            title_freq = (f"Phổ tần số (FFT) - Sample {current_sample_index_to_display} - File: {os.path.basename(mat_file_path)}\n"
                          f"(Key: {data_key}, {N} điểm, Tần số lấy mẫu: {sampling_rate} Hz, Bắt đầu từ điểm {actual_start_index} của tín hiệu gốc)")
            plt.title(title_freq)
            plt.xlabel("Tần số (Hz)")
            plt.ylabel("Biên độ")
            plt.grid(True)
            # Giới hạn trục x đến 2kHz hoặc tần số Nyquist, tùy cái nào nhỏ hơn
            plt.xlim([0, min(2000, sampling_rate / 2)])
            # Bạn có thể bỏ giới hạn này nếu muốn xem toàn bộ phổ:
            # plt.xlim([0, sampling_rate / 2])

            plt.tight_layout()
            plt.show()

    except FileNotFoundError:
        print(f"Lỗi: File không tìm thấy tại '{mat_file_path}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý file {os.path.basename(mat_file_path)}: {e}")
        import traceback
        traceback.print_exc()

# ==============================================================================
# PHẦN MAIN ĐỂ XỬ LÝ NHIỀU FILE
# ==============================================================================
if __name__ == "__main__":
    # --- Cấu hình ---
    base_data_directory = "CWRU-dataset-main/12k_Drive_End_Bearing_Fault_Data/"
    # Ví dụ: base_data_directory = "D:/CWRU_Data/12k_Drive_End/"

    files_to_process_with_fs = {
        "B/007/118_0.mat": 12000,   # Normal Baseline
        "B/014/185_0.mat" :12000,  # Inner Race Fault, 0.007", 0HP
        "B/021/222_0.mat":12000,  # Ball Fault, 0.007", 0HP
        "IR/007/105_0.mat":12000,  # Outer Race Fault (@6:00), 0.007", 0HP
        "IR/014/169_0.mat":12000,  # Inner Race Fault, 0.007", 1HP
        "IR/021/209_0.mat":12000,  # Ball Fault, 0.007", 1HP
        "OR/007/@3/144_0.mat":12000,  # Outer Race Fault (@6:00), 0.007", 1HP
        "OR/014/197@6_0.mat":12000,  # Inner Race Fault, 0.007", 2HP
        "OR/021/@3/246_0.mat":12000,  # Ball Fault, 0.007", 2HP
        "97_Normal_0.mat":12000,  # Outer Race Fault (@6:00), 0.007", 2HP
        # Thêm các file bạn muốn ở đây với tần số lấy mẫu tương ứng
        # Ví dụ cho dữ liệu 48kHz (nếu có):
        # "DE_Normal_0_48kHz.mat": 48000, # Tên file này là ví dụ, bạn cần tên file thực tế
        # "DE_IR007_0_48kHz.mat": 48000,
    }

    num_samples_to_draw_per_file = 5 # Vẽ FFT cho 2 sample đầu tiên của mỗi file
    sample_len = 4096

    # --- Xử lý các file ---
    print(f"Bắt đầu xử lý các file từ thư mục: '{os.path.abspath(base_data_directory)}'")
    print(f"Sẽ vẽ FFT cho {num_samples_to_draw_per_file} sample(s) đầu tiên cho mỗi file.\n")

    processed_files_count = 0
    failed_files_count = 0

    for file_name, fs in files_to_process_with_fs.items():
        full_file_path = os.path.join(base_data_directory, file_name)

        print(f"=============================================================")
        print(f"ĐANG XỬ LÝ FILE: {file_name} (Fs = {fs} Hz)")
        print(f"=============================================================")

        if os.path.exists(full_file_path):
            try:
                plot_cwru_fft_only( # Gọi hàm mới
                    mat_file_path=full_file_path,
                    sampling_rate=fs,
                    sample_length=sample_len,
                    num_samples_to_plot=num_samples_to_draw_per_file
                )
                print(f"\nHoàn tất xử lý và vẽ FFT cho file: {file_name}")
                processed_files_count += 1
            except Exception as e:
                print(f"Lỗi không mong muốn xảy ra khi gọi hàm cho {file_name}: {e}")
                failed_files_count +=1
        else:
            print(f"LỖI: File không tồn tại tại đường dẫn: {full_file_path}")
            print(f"Bỏ qua file: {file_name}\n")
            failed_files_count += 1
        print("-------------------------------------------------------------\n")

    print("=================== TỔNG KẾT ===================")
    print(f"Đã hoàn tất quá trình xử lý {len(files_to_process_with_fs)} file được chỉ định.")
    print(f"Số file được xử lý thành công: {processed_files_count}")
    print(f"Số file không tìm thấy hoặc gặp lỗi: {failed_files_count}")
    print("================================================")

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

# ==============================================================================
# HÀM ĐỂ TẠO ANIMATION FFT CHO MỘT FILE
# ==============================================================================
def animate_fft_for_file(mat_file_path, sampling_rate, output_gif_path,
                         sample_length=4096, num_samples_for_animation=10, fps=4):
    """
    Tải dữ liệu từ file .mat, tính FFT cho nhiều sample và tạo animation (GIF).

    Args:
        mat_file_path (str): Đường dẫn đến file .mat.
        sampling_rate (float): Tần số lấy mẫu (Hz).
        output_gif_path (str): Đường dẫn để lưu file GIF output.
        sample_length (int): Số điểm dữ liệu trong một sample.
        num_samples_for_animation (int): Số lượng sample sẽ được dùng để tạo animation.
        fps (int): Số frame trên giây cho GIF.
    """
    try:
        mat_data = scipy.io.loadmat(mat_file_path)
        file_basename = os.path.basename(mat_file_path)
        print(f"\n--- Tạo animation cho file: {file_basename} (Fs = {sampling_rate} Hz) ---")

        # (Logic tìm data_key và trích xuất time_series_data tương tự như trước)
        # ... (Phần này giữ nguyên)
        data_key = None
        keys_in_file = list(mat_data.keys())
        data_keys_info = {}
        for key in keys_in_file:
            if not key.startswith('__'):
                try:
                    data_array = mat_data[key]
                    if data_array.size > 1:
                        data_keys_info[key] = {'shape': data_array.shape, 'size': data_array.size}
                except Exception: pass

        priority_suffixes = ["_DE_time", "_FE_time", "_BA_time"]
        basename_no_ext_from_file = os.path.splitext(file_basename)[0] # Dùng file_basename cho X<ID>
        # Nếu file_name truyền vào có dạng "B/007/118_0.mat" thì basename_no_ext_from_file sẽ là "118_0"
        # Nhưng key trong file .mat thường là "X118_DE_time"
        # Chúng ta cần tên file gốc không có thư mục, ví dụ "118_0.mat"
        actual_file_id_part = os.path.splitext(os.path.basename(mat_file_path))[0]

        try:
            # Cố gắng lấy số từ tên file, ví dụ '118' từ '118_0'
            numeric_part_of_id = ''.join(filter(str.isdigit, actual_file_id_part.split('_')[0]))
            if numeric_part_of_id:
                 potential_file_based_keys = [f"X{int(numeric_part_of_id):03d}", f"X{actual_file_id_part}"]
            else: # Nếu không có phần số rõ ràng
                 potential_file_based_keys = [f"X{actual_file_id_part}"]
        except ValueError:
            potential_file_based_keys = [f"X{actual_file_id_part}"]


        for suffix in priority_suffixes:
            for k in data_keys_info.keys():
                if suffix in k: data_key = k; break
            if data_key: break
        if data_key is None:
            for pk in potential_file_based_keys:
                if pk in data_keys_info.keys(): data_key = pk; break
        if data_key is None:
            largest_key, max_size = None, 0
            for k, info in data_keys_info.items():
                if k.upper() != 'RPM' and info['size'] > max_size:
                    max_size, largest_key = info['size'], k
            if largest_key: data_key = largest_key
            else:
                print(f"Lỗi: Không tìm thấy key dữ liệu phù hợp trong {file_basename}.")
                return

        print(f"Đã chọn key dữ liệu: '{data_key}'")
        time_series_data = mat_data[data_key].flatten()
        total_data_points = len(time_series_data)

        if total_data_points == 0 or sample_length <= 0 or num_samples_for_animation <= 0:
            print("Dữ liệu không hợp lệ hoặc không có sample để tạo animation.")
            return

        all_time_samples = []
        num_possible_full_samples = total_data_points // sample_length
        actual_num_frames = 0

        if num_possible_full_samples == 0:
            if total_data_points > 0 :
                all_time_samples.append(time_series_data[:])
                actual_num_frames = 1
                print(f"Cảnh báo: Tín hiệu quá ngắn ({total_data_points} điểm), tạo animation với 1 frame (toàn bộ tín hiệu).")
        else:
            actual_num_frames = min(num_samples_for_animation, num_possible_full_samples)
            if num_samples_for_animation > num_possible_full_samples:
                print(f"Cảnh báo: Yêu cầu {num_samples_for_animation} frame, nhưng chỉ có {num_possible_full_samples} sample đầy đủ.")
            print(f"Sẽ tạo animation với {actual_num_frames} frame(s).")

            for i in range(actual_num_frames):
                start_idx = i * sample_length
                end_idx = start_idx + sample_length
                all_time_samples.append(time_series_data[start_idx:end_idx])

        if not all_time_samples:
            print(f"Không có đủ dữ liệu trong {file_basename} để tạo animation.")
            return

        max_overall_amplitude = 0.0
        processed_fft_data = []
        for i, time_sample_data_for_fft in enumerate(all_time_samples):
            N_current = len(time_sample_data_for_fft)
            if N_current == 0:
                processed_fft_data.append((np.array([]), np.array([])))
                continue
            window = np.hanning(N_current)
            y_windowed = time_sample_data_for_fft * window
            yf = np.fft.fft(y_windowed)
            xf_current_positive = np.fft.fftfreq(N_current, d=1/sampling_rate)[:N_current//2]
            yf_magnitude = (np.abs(yf[:N_current//2]) * (2.0/N_current))
            if N_current > 0: yf_magnitude[0] = np.abs(yf[0]) / N_current
            processed_fft_data.append((xf_current_positive, yf_magnitude))
            if len(yf_magnitude) > 0:
                current_max = np.max(yf_magnitude)
                if current_max > max_overall_amplitude:
                    max_overall_amplitude = current_max
        if max_overall_amplitude == 0.0 and actual_num_frames > 0:
             max_overall_amplitude = 0.1

        fig, ax = plt.subplots(figsize=(12, 6))
        line, = ax.plot([], [], lw=2)

        def init_animation():
            ax.set_xlim([0, min(2000, sampling_rate / 2)])
            ax.set_ylim([0, max_overall_amplitude * 1.1 if max_overall_amplitude > 0 else 0.1])
            ax.set_xlabel("Tần số (Hz)")
            ax.set_ylabel("Biên độ")
            ax.grid(True)
            line.set_data([], [])
            return line,

        def update_animation(frame_index):
            xf_positive, yf_magnitude_data = processed_fft_data[frame_index]
            line.set_data(xf_positive, yf_magnitude_data)
            start_point_of_sample = frame_index * sample_length
            N_for_title = len(all_time_samples[frame_index])
            ax.set_title(f"Phổ FFT - File: {file_basename} - Sample {frame_index}\n"
                         f"(Fs: {sampling_rate}Hz, Điểm bắt đầu: {start_point_of_sample}, {N_for_title} điểm)")
            return line,

        if actual_num_frames > 0:
            anim = FuncAnimation(fig, update_animation, frames=actual_num_frames,
                                 init_func=init_animation, blit=True, interval=1000/fps)
            try:
                # *** DÒNG SỬA LỖI CHÍNH Ở ĐÂY ***
                # Đảm bảo thư mục đích của file GIF tồn tại
                output_dir_for_gif = os.path.dirname(output_gif_path)
                if not os.path.exists(output_dir_for_gif) and output_dir_for_gif: # Kiểm tra output_dir_for_gif không rỗng
                    os.makedirs(output_dir_for_gif, exist_ok=True)
                    print(f"Đã tạo thư mục: {output_dir_for_gif}")
                # *** KẾT THÚC DÒNG SỬA LỖI ***

                anim.save(output_gif_path, writer='pillow', fps=fps)
                print(f"Đã lưu animation vào: {output_gif_path}")
            except Exception as e_save:
                print(f"Lỗi khi lưu animation: {e_save}.")
                print("Hãy đảm bảo Pillow (`pip install Pillow`) đã được cài đặt và đường dẫn hợp lệ.")
        else:
            print("Không có frame nào được tạo cho animation.")
        plt.close(fig)

    except FileNotFoundError:
        print(f"Lỗi: File không tìm thấy tại '{mat_file_path}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý file {os.path.basename(mat_file_path)}: {e}")
        import traceback
        traceback.print_exc()

# ==============================================================================
# PHẦN MAIN ĐỂ XỬ LÝ NHIỀU FILE
# ==============================================================================
if __name__ == "__main__":
    # --- Cấu hình ---
    # THAY ĐỔI ĐƯỜNG DẪN NÀY tới thư mục gốc chứa dữ liệu CWRU của bạn
    # Ví dụ: base_data_directory = "d:/CISLab/Code/CWRU-dataset-main/12k_Drive_End_Bearing_Fault_Data"
    # Hoặc nếu script nằm trong thư mục 'Code' và 'CWRU-dataset-main' cùng cấp:
    # base_data_directory = "../CWRU-dataset-main/12k_Drive_End_Bearing_Fault_Data"
    base_data_directory = "d:/CISLab/Code/CWRU-dataset-main/12k_Drive_End_Bearing_Fault_Data"


    # Thư mục lưu GIF, thư mục này sẽ được tạo trong thư mục chạy script (ví dụ: d:/CISLab/Code/fft_animations)
    output_animation_directory = "./fft_animations"
    # Nếu bạn muốn đường dẫn tuyệt đối:
    # output_animation_directory = "d:/CISLab/Code/fft_animations"
    os.makedirs(output_animation_directory, exist_ok=True) # Tạo thư mục cha cho tất cả các GIF

    # Cập nhật danh sách file bao gồm cả các thư mục con nếu cần
    # Ví dụ này giả sử file_name trong dictionary đã bao gồm đường dẫn tương đối từ base_data_directory
    files_to_process_with_fs = {
        "B/007/118_0.mat": 12000,
        "B/014/185_0.mat": 12000,
        "B/021/222_0.mat": 12000,
        "IR/007/105_0.mat": 12000,
        "IR/014/169_0.mat": 12000,
        "IR/021/209_0.mat": 12000,
        "OR/007/@6/130_0.mat": 12000,
        "OR/014/@6/197_0.mat": 12000,
        "OR/021/@6/234_0.mat": 12000,
        "Normal/97.mat": 12000, # File normal thường không có thư mục con B, IR, OR
        # Thêm các file và tần số lấy mẫu tương ứng
    }

    num_samples_for_anim_per_file = 20
    sample_len = 4096
    animation_fps = 5

    print(f"Bắt đầu xử lý các file từ thư mục: '{os.path.abspath(base_data_directory)}'")
    print(f"Sẽ tạo animation FFT với {num_samples_for_anim_per_file} frame(s) cho mỗi file (nếu có đủ dữ liệu).\n")

    for file_name_relative, fs_hz in files_to_process_with_fs.items():
        # full_file_path là đường dẫn tới file .mat
        full_file_path = os.path.join(base_data_directory, file_name_relative)

        # Tạo tên file GIF output, giữ nguyên cấu trúc thư mục con
        # file_name_relative_no_ext là "B/007/118_0"
        file_name_relative_no_ext = os.path.splitext(file_name_relative)[0]
        gif_sub_path_and_name = f"{file_name_relative_no_ext}_fft_anim_{num_samples_for_anim_per_file}frames.gif"
        full_gif_output_path = os.path.join(output_animation_directory, gif_sub_path_and_name)

        print(f"=============================================================")
        print(f"ĐANG XỬ LÝ FILE: {file_name_relative} (Đường dẫn đầy đủ: {full_file_path})")
        print(f"Tần số lấy mẫu: {fs_hz} Hz")
        print(f"GIF output dự kiến: {full_gif_output_path}")
        print(f"=============================================================")

        if os.path.exists(full_file_path):
            animate_fft_for_file(
                mat_file_path=full_file_path,
                sampling_rate=fs_hz,
                output_gif_path=full_gif_output_path,
                sample_length=sample_len,
                num_samples_for_animation=num_samples_for_anim_per_file,
                fps=animation_fps
            )
        else:
            print(f"LỖI: File không tồn tại: {full_file_path}. Bỏ qua.\n")
        print("-------------------------------------------------------------\n")

    print("=================== HOÀN TẤT ===================")
    print(f"Quá trình tạo animation cho {len(files_to_process_with_fs)} file đã kết thúc.")
    print(f"Các file GIF (nếu có) được lưu trong thư mục: '{os.path.abspath(output_animation_directory)}'")
    print("================================================")